# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [20]:
from queue import PriorityQueue
import copy
from numpy import random
random.seed(151936)

In [21]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [22]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [23]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [24]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [25]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [26]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [27]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [28]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [29]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.assignment = {}
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable):
        return self.csp.domains[variable]
        
    
    def solve(self):
        if self.csp.is_complete(self.assignment):
            return self.assignment

        var = self.select_unassigned_variable(self.assignment)
        for value in self.order_domain_values(var):
            self.counter += 1
            self.assignment[var] = value
            if self.csp.is_consistent(self.assignment):
                self.assignment[var] = value
                self.solve()
                if self.csp.is_complete(self.assignment):
                    return self.assignment
            self.assignment.pop(var)
        return False

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [40]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'R', 'NT': 'B', 'SA': 'G', 'Q': 'R', 'NSW': 'B', 'V': 'R', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [31]:
class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.assignment = {}
        self.counter = 0

    def select_unassigned_variable(self, assignment, domains):
        memory = (None, 'legalValues', -1)
        for var in self.csp.domains.keys():
            legalValues = domains[var]
            degree = 0

            for constraint in self.csp.binary:
                if var in constraint:
                    if constraint[2 - constraint.index(var)] in self.assignment.keys():
                        if self.assignment[constraint[2 - constraint.index(var)]] in legalValues:
                            legalValues.remove(self.assignment[constraint[2 - constraint.index(var)]])
                    if constraint[2 - constraint.index(var)] not in self.assignment.keys():
                        degree += 1

            if var not in assignment and (memory[1] == 'legalValues' or len(legalValues) <= memory[1]):
                if memory[1] == 'legalValues':
                    memory = (var, len(legalValues), degree)
                elif len(legalValues) < memory[1]:
                    memory = (var, len(legalValues), degree)
                else:
                    if degree > memory[2]:
                        memory = (var, len(legalValues), degree)

        return memory[0]
    
    def order_domain_values(self, variable, domains):
        return domains[variable]
        
    def solve(self, domains):
        if domains == None:
            domainsCopy = copy.deepcopy(self.csp.domains)
        else:
            domainsCopy = copy.deepcopy(domains)

        if self.csp.is_complete(self.assignment):
            return self.assignment

        var = self.select_unassigned_variable(self.assignment, domainsCopy)
        for value in self.order_domain_values(var, domainsCopy):
            domainsCopy2 = copy.deepcopy(domainsCopy)
            self.counter += 1
            self.assignment[var] = value        
            if self.csp.is_consistent(self.assignment):
                self.assignment[var] = value
                domainsCopy2[var] = {value}
                self.solve(domainsCopy2)
                if self.csp.is_complete(self.assignment):
                    return self.assignment
            self.assignment.pop(var)
        return False
        ...

Lets test your implementation by solving the problem of coloring Australia!

In [32]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve(None)
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'R', 'NT': 'B', 'Q': 'G', 'NSW': 'B', 'WA': 'G', 'V': 'G', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

Heuristic allows the algorithm to choose variables that are more certain to yield good results and thus does not need to back track that much as opposed to choosing variables with a lot of values where chosing the correct one is way less probable and might result in a lot of uneccesary backtracking, that why heuristic backtracking has less considered assignments than normal recusive backtracking.

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [33]:
puzzles = ['''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''
,
'''
8________
__36_____
_7__9_2__
_5___7___
____457__
___1___3_
__1____68
__85___1_
_9____4__
'''
,
'''
____1__2_
2__3____8
___8_245_
8_32__7__
_________
__6__53_2
_376_8___
4____1__9
_9__3____
'''
,
'''
4_____6__
_7_3_____
_2_5_1___
_9___682_
3_______9
_489___5_
___7_5_1_
_____4_8_
__7_____2
'''
,
'''
_5_8__3__
_48_2____
_9_6_5___
8___7_5__
7_______9
__9_3___2
___3_1_8_
____6_41_
__3__8_5_
''']

In [34]:
solution1 = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [35]:
def setUpSudoku(puzzle):
    sudoku = CSP()
    for i in range(len(puzzle)):
        if puzzle[i] == '_' :
            sudoku.addVariable((1 + i//10, i%10),{str(i) for i in range(1, 10)})
        elif puzzle[i] != '\n':
            sudoku.addVariable((1 + i//10, i%10),{puzzle[i]})

    for i in range(len(puzzle)):
        row = 1 + i//10
        column = i%10
        for j in range(1, 10):
            if column != j and puzzle[i] != '\n' and ((row, j), '!=', (row, column)) not in sudoku.binary:
                sudoku.addBinaryConstraint((row, column), '!=', (row, j))

            if row != j and puzzle[i] != '\n' and ((j, column), '!=', (row, column)) not in sudoku.binary:
                sudoku.addBinaryConstraint((row, column), '!=', (j, column))
            
        if puzzle[i] != '\n':
            row -= 1
            column -= 1
            for k in range(row-row%3, row-row%3 + 3):
                for l in range(column-column%3, column-column%3 + 3):
                    r = k + 1
                    c = l + 1
                    if r <= 9 and c <= 9 and ((r, c), '!=', (row + 1, column + 1)) not in sudoku.binary and ((row + 1, column + 1), '!=', (r, c)) not in sudoku.binary and (r, c) != (row + 1, column + 1):
                        sudoku.addBinaryConstraint((row + 1, column + 1), '!=', (r, c))
    return sudoku

                    
def drawSudoku(assignment):
    print("\n-------------------------")

    for i in range(1, 10):
        for j in range(1, 10):
            if assignment[(i, j)] is not None:
                if j == 0:
                    print("|", end=" ")
                print(f"{assignment[(i, j)]} ", end="")
            if (j) % 3 == 0:
                print("|", end=" ")
        if (i) % 3 == 0:
            print("\n-------------------------", end=" ")
        print()

def reverseOperator(op):
    if op[0] == '=':
        return '='
    if op == '!=':
        return '!='
    if op == '<':
        return '>'
    if op == '<=':
        return '>='
    if op == '>':
        return '<'
    if op == '>=':
        return '<='

Ellipsis

Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [36]:
sudoku = setUpSudoku(puzzles[0])
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve(None)
print("Assignment", assignment)
drawSudoku(assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

Assignment {(1, 3): '3', (2, 4): '3', (3, 7): '4', (4, 6): '2', (5, 1): '7', (8, 9): '9', (9, 5): '1', (6, 6): '8', (7, 3): '2', (1, 7): '6', (3, 4): '8', (5, 9): '8', (2, 1): '9', (9, 3): '5', (4, 4): '1', (6, 7): '2', (7, 6): '9', (1, 5): '2', (8, 1): '8', (2, 9): '1', (3, 3): '1', (4, 7): '9', (6, 4): '7', (8, 6): '3', (7, 7): '5', (4, 3): '8', (9, 7): '3', (5, 7): '1', (2, 6): '5', (6, 3): '6', (7, 4): '6', (3, 6): '6', (8, 4): '2', (5, 6): '4', (5, 3): '9', (8, 7): '7', (7, 9): '4', (9, 4): '4', (9, 1): '6', (8, 5): '5', (9, 9): '2', (9, 8): '8', (7, 8): '1', (7, 1): '3', (7, 2): '7', (9, 2): '9', (1, 4): '9', (3, 5): '7', (2, 5): '4', (2, 3): '7', (2, 8): '2', (8, 8): '6', (1, 6): '1', (2, 7): '8', (2, 2): '6', (5, 4): '5', (5, 8): '3', (5, 2): '2', (3, 2): '5', (6, 9): '5', (6, 8): '4', (1, 1): '4', (4, 1): '5', (1, 2): '8', (3, 8): '9', (3, 9): '3', (4, 8): '7', (6, 1): '1', (7, 5): '8', (4, 9): '6', (4, 5): '3', (6, 2): '3', (1, 8): '5', (4, 2): '4', (5, 5): '6', (8, 2): '1', 

## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [37]:
class RecursiveBacktrackingWithAC3:
    def __init__(self, csp):
        self.csp = csp
        self.assignment = {}
        self.counter = 0

    def select_unassigned_variable(self, assignment, domains):
        memory = (None, 'legalValues', -1)
        for var in self.csp.domains.keys():
            legalValues = domains[var]
            degree = 0

            for constraint in self.csp.binary:
                if var in constraint:
                    if constraint[2 - constraint.index(var)] in self.assignment.keys():
                        if self.assignment[constraint[2 - constraint.index(var)]] in legalValues:
                            legalValues.remove(self.assignment[constraint[2 - constraint.index(var)]])

                    if constraint[2 - constraint.index(var)] not in self.assignment.keys():
                        degree += 1

            if var not in assignment and (memory[1] == 'legalValues' or len(legalValues) <= memory[1]):
                if memory[1] == 'legalValues':
                    memory = (var, len(legalValues), degree)
                elif len(legalValues) < memory[1]:
                    memory = (var, len(legalValues), degree)
                else:
                    if degree > memory[2]:
                        memory = (var, len(legalValues), degree)

        return memory[0]
    
    def order_domain_values(self, variable, domains):
        return domains[variable]

    def removeInconsistentValues(self, domain, Xi, op, Xj):
        removed = False
        memory = []
        for x in domain[Xi]:
            temp = 0
            for y in domain[Xj]:
                if self.csp.verify(x, op, y):
                    temp += 1
            if temp == 0:
                removed = True
                memory.append(x)
        for x in memory:
            domain[Xi].remove(x)
        return removed


    def AC3(self, domain):
        queue = [i for i in self.csp.binary]
        for Xi, op, Xj in self.csp.binary:
            queue.append((Xj, reverseOperator(op), Xi))

        while len(queue) != 0:
            Xi, op, Xj = queue.pop(0)
            if self.removeInconsistentValues(domain, Xi, op, Xj):
                for constraint in self.csp.binary:
                    if Xi in constraint:
                        queue.append(constraint)
        
    def solve(self, domains):
        if domains == None:
            domainsCopy = copy.deepcopy(self.csp.domains)
        else:
            domainsCopy = copy.deepcopy(domains)

        self.AC3(domainsCopy)
        

        if self.csp.is_complete(self.assignment):
            return self.assignment

        var = self.select_unassigned_variable(self.assignment, domainsCopy)

        for value in self.order_domain_values(var, domainsCopy):
            domainsCopy2 = copy.deepcopy(domainsCopy)
            self.counter += 1
            self.assignment[var] = value
            if self.csp.is_consistent(self.assignment):
                self.assignment[var] = value
                domainsCopy2[var] = {value}
                self.solve(domainsCopy2)
                if self.csp.is_complete(self.assignment):
                    return self.assignment
            self.assignment.pop(var)
        return False

Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [38]:
solver = RecursiveBacktrackingWithAC3(australia)
assignment = solver.solve(None)
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'R', 'NT': 'B', 'NSW': 'B', 'WA': 'G', 'Q': 'G', 'V': 'G', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


In [39]:
sudoku = setUpSudoku(puzzles[0])
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve(None)
print("Assignment", assignment)
drawSudoku(assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {(1, 3): '3', (2, 4): '3', (3, 7): '4', (4, 1): '5', (5, 5): '6', (6, 8): '4', (7, 2): '7', (8, 6): '3', (9, 9): '2', (1, 1): '4', (2, 5): '4', (3, 8): '9', (4, 2): '4', (5, 4): '5', (6, 7): '2', (7, 3): '2', (8, 9): '9', (9, 6): '7', (1, 2): '8', (2, 6): '5', (3, 9): '3', (4, 3): '8', (7, 1): '3', (5, 6): '4', (6, 9): '5', (8, 4): '2', (9, 7): '3', (8, 5): '5', (9, 8): '8', (1, 4): '9', (2, 1): '9', (4, 5): '3', (5, 2): '2', (1, 7): '6', (2, 3): '7', (3, 4): '8', (4, 8): '7', (6, 2): '3', (7, 6): '9', (5, 8): '3', (6, 5): '9', (7, 9): '4', (8, 1): '8', (2, 7): '8', (3, 3): '1', (9, 1): '6', (1, 5): '2', (4, 4): '1', (5, 3): '9', (7, 7): '5', (8, 2): '1', (1, 6): '1', (2, 8): '2', (3, 1): '2', (4, 9): '6', (6, 3): '6', (9, 2): '9', (5, 9): '8', (6, 4): '7', (7, 5): '8', (8, 7): '7', (1, 8): '5', (3, 6): '6', (9, 5): '1', (2, 2): '6', (4, 6): '2', (5, 1): '7', (7, 4): '6', (8, 3): '4', (1, 9): '7', (3, 2): '5', (4, 7): '9', (6, 1): '1', (7, 8): '1', (9, 3): '5', (2, 9): '1', 

------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.